In [2]:
import pandas as pd
import ttach as tta

In [3]:
# !pip install optuna
# !pip install --upgrade numpy

In [4]:
import os
import shutil
import pathlib
import gc
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

In [5]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [6]:
seed_everything(42)

In [7]:
CFGS = [           
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth': '../../output/exp_pl2_new01/Unet/efficientnet-b7-256/checkpoint_dice.pth',
#         'threshold': 0.84,
#         'tta': True, 
#     },
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.9,
        'tta': True, 
    },
    
]

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [9]:
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
#         self.tta = tta_count
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
#         if  self.transform :
#             test_aug = self.transform(image=image, mask=label)
#             augmented_img  = test_aug['image']
#             label  = test_aug['mask']
            
#             if self.tta > 1:
#                 images = []
#                 images.append(augmented_img)
#                 images.append(self.transform(image=np.fliplr(image))['image'])
#                 if self.tta > 2:
#                     images.append(self.transform(image=np.flipud(image))['image'])
#                 if self.tta > 3:
#                     images.append(self.transform(image=np.flipud(np.fliplr(image)))['image'])
#                 image = np.stack(images, axis=0)
# #                 print(image.shape )
#                 image = np.transpose(image, (0,3,1,2))
# #                 print(image.shape)
# #                 print("*"*100)
#             else:
#                 image = augmented_img
#                 image = np.transpose(image, (2, 0, 1))
                        
#             label = np.transpose(label, (2, 0, 1))
            
        return torch.tensor(image), torch.tensor(label)

In [10]:
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=cfg["num_classes"],        # model output channels (number of classes in your dataset)
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [11]:
import torch
import torch.nn as nn


class Dice(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred_masks: torch.Tensor, gt_masks: torch.Tensor, smooth=1e-5):
        """
        This method expect boolean pred and gt masks in the shape of (B, C, H, W).
        In this competition, we are dealing with semantic segmentation and I choose to use 
        one channel with two values: 0 for background, 1 for mask.
        """
        batch_size = pred_masks.shape[0]
        pred_masks = pred_masks.view(batch_size, -1)
        gt_masks = gt_masks.view(batch_size, -1)

        intersection = (pred_masks * gt_masks).sum(1)
        dice = (2. * intersection) / (pred_masks.sum(1) + gt_masks.sum(1) + smooth)

        dice = dice.mean()

        return dice

In [12]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice



def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform



def post_process_minsize(mask, min_size=10):
    '''Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored'''
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros(mask.shape, np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num


In [13]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
# val_df = val_df.loc[val_df['class'] == 'tensor(1)'].reset_index(drop=True)
# train_df = pd.read_csv("../../input/data_utils/train_5_folds.csv")
# val_df = train_df.loc[train_df.fold == 0].reset_index(drop=True)



final_preds = []

for idx, cfg in enumerate(CFGS):
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    model = Net(cfg)    
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    if cfg['tta']:
        model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

        
    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest')             
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():            
            pred = model(images)                
            pred = pred.sigmoid()
        
            if cfg['img_size'][0] != 256:
                pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
            preds.append(torch.squeeze(pred, dim=1))

    
#     model_masks = torch.stack(masks_, dim=0)
#     model_preds = torch.stack(preds, dim=0)
    
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
    
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    dice_score = dice_coef(model_masks, model_preds).cpu().detach().numpy() 
    dice_score_th = dice_coef(model_masks, model_preds, thr=cfg['threshold']).cpu().detach().numpy() 
        
        
        
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :        
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
        
        
#     print(dice_score, dice_score_th)
    final_preds.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    # 0.65424037

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [384, 384], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.9, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.6749351 0.33


In [41]:
def boost_dice(params):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
#     threshold = 0.2
#     preds1 = (nn.Sigmoid()(preds)>threshold).double()            
#     score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
    best_threshold = 0.0
    best_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         if threshold < 0.05 and threshold > 0.25:
#             continue
    
        preds1 = (nn.Sigmoid()(preds)>threshold).double()            
        score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [ ]:
def objective(trial):
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)  
        
        
    score, best_threshold  = boost_dice(params)
    params['threshold'] = best_threshold
    print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

In [ ]:
best_params

In [55]:
x = torch.ones([32,3,3,256,256])

In [56]:
x.shape

torch.Size([32, 3, 3, 256, 256])

In [57]:
for a in [x[:, 0,:,:,:], x[:, 1,:,:,:]]:
    print(a.shape)

torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])


In [ ]:
pred = torch.mean(torch.stack(pred_tta), dim=0)            
